## Etapa atual
- **Validação da ideia**: criação de uma versão em excel para comparação de preços mercados fisicos.  
- **Criação de base inicial**: coletar dados de algumas notas fiscais ou produtos.  
- **Objetivo desta etapa**: ter **uma base inicial de preços** para análises e comparativos.

##  Estrutura para etapa atual
1. Criar **script para raspar dados das notas fiscais**:  
   - Extrair: produto, preço, mercado, data.  
2. Alimentar **planilha ou banco de dados simples** (CSV / Google Sheets / SQLite).  
3. Fazer **comparativo de preços iniciais** usando esses dados.  
4. Validar se o MVP manual fornece resultados confiáveis antes de automatizar scraping de sites ou QR Codes.


In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


In [34]:
# SCRIPT PARA RASPAGEM DE DADOS


links = [
    "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250906057223036281650260000081679261712319|3|1|29|883.81|||MNrdIOQPqR9rf+0Ml7ylMFDIQmbci+OHfherKlt/SZ8gtmM6/cuUDrtfsFaPKVCSdAP5HQK5BbeAyJ9LXt1FD4WAlNwW0r5i8pV8+agHcEbSJWG1R1ikq0wxsIQlsROl+TMRj7BQy2ocU8zxOl8IiV9ubPBC/NhMBuaEm6MMbcbNtmfBhXfGylmrLIPmc35vIlgA6XvV1tTZpUq7UEe+4dLn+rJ1b8yRLSDJjpaft0XETpVKduuMICvMuFU1wUgu3PlKsW9b5TtwP0/1Uq1kay+uhicezK7MJ272EIZ+TDOg4XVImZQqVIFCY/jToSCe1ROwj6JMOGNhs6WQjjlLWA=="
]
def cadastro_nome_produto(dados):
    tit_produto = dados.find_all('span', class_='txtTit')
    nomes_produtos = [p.get_text(strip=True) for p in tit_produto]
    return nomes_produtos

def cadastro_quantidade_comprada(dados):
    quantidade = dados.find_all('span', class_='Rqtd')
    quantidade_comprada = [p.get_text(strip=True).replace("Qtde.:", "") for p in quantidade]
    return quantidade_comprada

def cadastro_unidade_medida(dados):
    unidade = dados.find_all('span', class_='RUN')
    unidade_produto = [p.get_text(strip=True).replace("UN:", "") for p in unidade]
    return unidade_produto

def cadastro_valor_unitario(dados):
    valor_unitario = dados.find_all('span', class_='RvlUnit')
    vl_unitario = [
        p.get_text(strip=True).replace("Vl. Unit.:", "").replace(",", ".")
        for p in valor_unitario
    ]
    return vl_unitario

def cadastro_valor_total(dados):
    valor_total = dados.find_all('td', class_='txtTit noWrap')
    vl_total = [
        p.get_text(strip=True).replace("Vl. Total", "").replace(",", ".")
        for p in valor_total
    ]
    return vl_total

def cadastro_nome_mercado(dados):
    nome_empr = dados.find_all('div', class_='txtTopo')
    nome_empresa = [p.get_text(strip=True) for p in nome_empr]
    return nome_empresa

def cadastro_cnpj_empresa(dados): 
    documento = dados.find_all('div', class_='text')
    cnpjs = []
    for p in documento:
        texto = p.get_text(strip=True)
        if "CNPJ:" in texto:  # garante que só pega o bloco do CNPJ
            cnpjs.append(texto.replace("CNPJ:", "").strip())
    return cnpjs


def cadastro_data_emissao(dados):
    tag = dados.find("strong", string=lambda s: s and "Emissão" in s)
    if tag:
        data_texto = tag.next_sibling.strip()
        data_emissao = data_texto.split("-")[0].strip()
        return data_emissao
    return None


def cria_dataframe(dados):
    # usa as funções que você já criou
    cnpj = cadastro_cnpj_empresa(dados)
    nome_empresa = cadastro_nome_mercado(dados)
    nomes_produtos = cadastro_nome_produto(dados)
    quantidade_comprada = cadastro_quantidade_comprada(dados)
    unidade_produto = cadastro_unidade_medida(dados)
    vl_unitario = cadastro_valor_unitario(dados)
    vl_total = cadastro_valor_total(dados)
    data_emissao = cadastro_data_emissao(dados)

    # garante que os dados "fixos" repitam para cada produto
    cnpj = [cnpj[0]] * len(nomes_produtos)
    nome_empresa = [nome_empresa[0]] * len(nomes_produtos)
    data_emissao = [data_emissao] * len(nomes_produtos)

    # monta dicionário
    dados_dicionario = {
        'CNPJ': cnpj,
        'Empresa': nome_empresa,
        'Data Emissão': data_emissao,
        'Produto': nomes_produtos,
        'Qtde': quantidade_comprada,
        'UN': unidade_produto,
        'Vl. Unit.': vl_unitario,
        'Vl. Total': vl_total
    }

    # cria dataframe
    df = pd.DataFrame(dados_dicionario)

    # converte valores numéricos
    df['Qtde'] = df['Qtde'].str.replace(",", ".").astype(float)
    df['Vl. Unit.'] = df['Vl. Unit.'].astype(float)
    df['Vl. Total'] = df['Vl. Total'].astype(float)

    return df

dfs = []

for url in links:
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            df_nfe = cria_dataframe(soup)
            dfs.append(df_nfe)
        else:
            print(f"Erro ao acessar {url}: {response.status_code}")
    except Exception as e:
        print(f"Erro ao processar {url}: {e}")

# Concatenar todas as notas em um único DataFrame
df_final = pd.concat(dfs, ignore_index=True)

# Salvar em CSV
df_final.to_excel("notas_raspadas.xlsx", index=False)

In [ ]:
df_final

0      1.0
1      1.0
2      1.0
3      1.0
4      1.0
      ... 
79     1.0
80     1.0
81     1.0
82    12.0
83     1.0
Name: Qtde, Length: 84, dtype: float64